In [1]:
import cv2
import numpy as np
import pandas as pd 
import face_recognition
import os
import supervision as sv
from datetime import datetime
from ultralytics import YOLOv10


In [2]:
pwd

'H:\\Code-for-my-Courses-GABER-main\\child detection in car'

In [3]:
cd "H:\Code-for-my-Courses-GABER-main\attendace sytem"

H:\Code-for-my-Courses-GABER-main\attendace sytem


In [4]:
pwd

'H:\\Code-for-my-Courses-GABER-main\\attendace sytem'

In [5]:
ls

 Volume in drive H is gaber
 Volume Serial Number is 52D0-1D8A

 Directory of H:\Code-for-my-Courses-GABER-main\attendace sytem

10/02/2024  02:17 AM    <DIR>          .
10/02/2024  02:17 AM    <DIR>          ..
07/08/2024  12:27 PM    <DIR>          .ipynb_checkpoints
07/08/2024  06:01 PM            47,253 Ahmed Gaber.jpeg
10/01/2024  02:20 PM                47 Attendance.csv
10/01/2024  01:50 PM    <DIR>          AttendanceImages
10/06/2024  02:29 PM                94 attendce_rec.csv
07/06/2024  11:49 PM         5,760,164 fakeVSreal.pt
07/08/2024  06:05 PM            56,348 Mohammed Gaber.JPG
07/12/2024  04:57 AM               617 Untitled.ipynb
07/08/2024  11:53 AM    <DIR>          yolov10
               6 File(s)      5,864,523 bytes
               5 Dir(s)  52,099,264,512 bytes free


In [6]:
path = 'AttendanceImages'
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['Ahmed Gaber.JPG', 'Mohammed Gaber.JPG']


In [7]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['Ahmed Gaber', 'Mohammed Gaber']


In [8]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [9]:
df=pd.DataFrame(columns=(["Present name","Attendance time","Absent name"]))

In [10]:
df.to_csv("attendce_rec.csv",index=False)

In [11]:
def markAttendance(name):
    df = pd.read_csv("attendce_rec.csv")
    names=[]
    [names.append(n.upper())for n in classNames]
    absent_names=[]
    # Check if the name is already in the DataFrame
    if name not in df["Present name"].values:
        
        names.remove(name)
        now = datetime.now()
        # Create a new row by passing values as lists
        new_row = pd.DataFrame({"Present name": [name], "Attendance time": [now.strftime("%d/%m/%Y %H:%M:%S")]})
        df = pd.concat([df, new_row], ignore_index=True)
        
    
    # Create a new row for "Absent name"
    [ absent_names.append(nam) for nam in names if nam not in df["Absent name"].values and nam not in df["Present name"].values ]
    new_row = pd.DataFrame({"Absent name": absent_names})
    df = pd.concat([df, new_row], ignore_index=True)
    df.to_csv("attendce_rec.csv", index=False)

# Example usage
encodeListKnown = findEncodings(images)
print('Encoding Complete')




Encoding Complete


In [12]:
import cv2
import torch
import numpy as np
import face_recognition

# Assuming YOLOv10, sv.BoundingBoxAnnotator, sv.LabelAnnotator, and other dependencies are already defined/imported

model = YOLOv10('fakeVSreal.pt')

bounding_box_annotator = sv.BoundingBoxAnnotator()

label_annotator = sv.LabelAnnotator()
cap = cv2.VideoCapture(0)
frame_resizing = 0.25

while True:
    success, img = cap.read()
    if not success:
        break
    
    results = model(img, conf=0.75)[0]
    detections = sv.Detections.from_ultralytics(results)


    if results.boxes.cls.numel() == 0:
        print("No classes detected.")
    else:
        # Flags to check presence of both classes
        real_detected = (results.boxes.cls == 1).any()
        fake_detected = (results.boxes.cls == 0).any()
        
        if real_detected:
            imgS = cv2.resize(img, (0, 0), fx=frame_resizing, fy=frame_resizing)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
            
            facesCurFrame = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
            
            for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
                matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
                faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
                matchIndex = np.argmin(faceDis)
                if matches[matchIndex]:
                    name = classNames[matchIndex].upper()
                    faceLoc = np.array(faceLoc)
                    faceLoc = faceLoc / frame_resizing
                    faceLoc = faceLoc.astype(int)                    
                    y1, x2, y2, x1 = faceLoc[0], faceLoc[1], faceLoc[2], faceLoc[3]
                    cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                    markAttendance(name)

        if fake_detected :
            detections = sv.Detections.from_ultralytics(results)
            annotated_image = bounding_box_annotator.annotate(
                scene=img, detections=detections)
            annotated_image = label_annotator.annotate(
                scene=annotated_image, detections=detections ,)
            img = annotated_image
    
    cv2.imshow('Webcam', img)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 334.2ms
Speed: 20.9ms preprocess, 334.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
No classes detected.

0: 480x640 (no detections), 302.0ms
Speed: 3.5ms preprocess, 302.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
No classes detected.

0: 480x640 (no detections), 294.7ms
Speed: 2.0ms preprocess, 294.7ms inference, 8.1ms postprocess per image at shape (1, 3, 480, 640)
No classes detected.

0: 480x640 (no detections), 303.1ms
Speed: 0.0ms preprocess, 303.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
No classes detected.

0: 480x640 (no detections), 304.5ms
Speed: 5.8ms preprocess, 304.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
No classes detected.

0: 480x640 (no detections), 294.4ms
Speed: 4.1ms preprocess, 294.4ms inference, 8.1ms postprocess per image at shape (1, 3, 480, 640)
No classes detected.

0: 480x640 (no detections), 301.5ms
Speed: 0.0ms preprocess, 3